# Showcase: Intro to Fixed Income with BQL


This notebook is designed to show off a variety of ways you can use BQL to perform fixed income analyses.


Click **Run all** (<i class="fas fa-forward"></i>) at the top of this window to run all the cells in this notebook.


**Table of contents**

- [Setting up your environment](#bql-for-fi-setup)
- [Querying BQL for financial data](#bql-for-fi-querying)
  - [Querying for an individual bond](#bql-for-fi-querying-bond)
  - [Querying for multiple data items](#bql-for-fi-querying-data)
  - [Querying for multiple bonds](#bql-for-fi-querying-multiple)
  - [Specifying dates](#bql-for-fi-querying-dates)
  - [Retrieving times series data](#bql-for-fi-querying-time-series)
- [Building universes with `univ`](#bql-for-fi-univ)
  - [Building a universe from a list of bonds](#bql-for-fi-univ-from-list)
  - [Building a universe from an existing basket of securities](#bql-for-fi-univ-from-basket)
  - [Specifying capital structure issuers](#bql-for-fi-univ-cap-structure)
  - [Determining index membership at a point in time](#bql-for-fi-univ-pit)
  - [Importing private portfolios from the Bloomberg Terminal®](#bql-for-fi-univ-import)
- [Aggregating, ranking, and sorting data](#bql-for-fi-aggregating)
  - [Performing aggregate analyses with `group()`](#bql-for-fi-aggregating-group)

- [Conclusion](#bql-for-fi-conclusion)


---

<a id="bql-for-fi-setup" />


## Setting up your environment


Before you can run the examples in this notebook, you must first set up your environment by importing the necessary libraries (`pandas` and `bql`) and connecting to the BQL service:

1. Import pandas if needed for DataFrame handling.
2. Import Bloomberg's PyBQL library, named `bql`.
3. Connect to the BQL Service by creating an instance of the `bql.Service()` class. This allows you to access BQL data items, functions, and universe functions, as well as generate and execute requests. It's recommended to assign the instance to a variable; bq is the common convention.

In [29]:
# Set up your environment
import pandas as pd
import bql

# Connect to the BQL service
bq = bql.Service()


---

<a id="bql-for-fi-querying" />


## Querying BQL for data

After this setting up your environment, you're ready to retrieve BQL data using the following steps:

1. Define your target universe and target data.
2. Create a [`bql.Request`](https://help.bquant.blpprofessional.com/content?id=B2HrxNiSiuKvhYfrCxMfU5) instance with your target universe and target data as arguments.
3. Use the [`bq.execute()`](https://help.bquant.blpprofessional.com/content?id=5CA4pAL4KAoMi3LayCYLqr#bql\.Service\.execute)  function to send the request to the BQL Service.
4. Print the response as a DataFrame to see the output.

There are 3 main types of BQL objects you can use to build a query: 

<table style="width: 100%; border-collapse: collapse; border: solid rgb(0, 0, 0);">
    <tbody>
        <tr>
            <td style="width: 15%; border: 3px solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">BQL Object</span></strong></div>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">PyBQL Property</span></strong></div>
            <td style="width: 50%; border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Action</span></strong></div>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Example</span></strong></div>
        </tr>
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Universe function</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);"><span style='background-color:gray'>univ</span></td>
            <td style="width: 50%; border: solid rgb(0, 0, 0);">Set criteria to create a target universe of entities.</td>
            <td style="width: 20%; border: solid rgb(0, 0, 0);"><span style='background-color:gray'>bq.univ.members()</span></td>
        </tr>
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Data item</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);"><span style='background-color:gray'>data</span></td>
            <td style="width: 50%; border: solid rgb(0, 0, 0);">Retrieve a specific data point (e.g., a price) for an entity.</td>
            <td style="width: 20%; border: solid rgb(0, 0, 0);"><span style='background-color:gray'>bq.data.px_last()</span></td>
        </tr>
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Function</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);"><span style='background-color:gray'>func</span></td>
            <td style="width: 50%; border: solid rgb(0, 0, 0);">Perform a calculation or transformation on a data point.</td>
            <td style="width: 20%; border: solid rgb(0, 0, 0);"><span style='background-color:gray'>bq.func.avg()</span></td>
        </tr>
    </tbody>
</table>



---

<a id="bql-for-fi-querying-bond" />


### Querying for an individual bond

This example constructs and executes a BQL query to get the name of an individual bond.


In [30]:
# Set the universe as an individual bond, identified by its Bloomberg ticker
universe = 'ZS7759443 Corp'

# Query BQL for the bond's short name
data_item = bq.data.name()

# Build and execute the BQL query
request = bql.Request(universe, data_item)
response = bq.execute(request)

# Display the data as a Pandas DataFrame
data = response[0].df()
data

,NAME()
ID,
ZS7759443 Corp,VOD 1 ⅝ 11/24/30


---

<a id="bql-for-fi-querying-data" />


### Basic market data and parameters

<table style="width: 100%; border-collapse: collapse; border: solid rgb(0, 0, 0);">
    <tbody>
        <tr>
            <td style="width: 15%; border: 3px solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Field Name</span></strong></div>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Mnenomic</span></strong></div>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Example</span></strong></div>
        </tr>
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Price</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">PX_LAST</td>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">bq.data.px_last()</td>
        </tr>
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Spread</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">SPREAD</td>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">bq.data.spread()</td>
        </tr>
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Yield</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">YIELD_</td>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">bq.data.yield_()</td>
        </tr>
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Duration</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">DURATION</td>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">bq.data.duration()</td>            
        </tr>
    </tbody>
</table>


<table style="width: 100%; border-collapse: collapse; border: solid rgb(0, 0, 0);">
    <tbody>
        <tr>
            <td style="width: 12.5%; border: 3px solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Name</span></strong></div>
            <td style="width: 12.5%; border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Mnenomic</span></strong></div>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Default input</span></strong></div>
            <td style="width: 35%;border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Available inputs</span></strong></div>
            <td style="width: 25%;border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Example</span></strong></div>
        </tr>
        <tr>
            <td style="width: 12.5%; border: solid rgb(0, 0, 0);">Pricing Source</td>
            <td style="width: 12.5%; border: solid rgb(0, 0, 0);">PRICING_SOURCE</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">BVAL</td>
            <td style="width: 35%; border: solid rgb(0, 0, 0);">Pricing sources available on ALLQ</td>
            <td style="width: 25%; border: solid rgb(0, 0, 0);">bq.data.yield_(pricing_source='BGN')</td>
        </tr>
        <tr>
            <td style="width: 12.5%; border: solid rgb(0, 0, 0);">Side</td>
            <td style="width: 12.5%; border: solid rgb(0, 0, 0);">SIDE</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">BID</td>
            <td style="width: 35%; border: solid rgb(0, 0, 0);">BID, MID, ASK</td>
            <td style="width: 25%; border: solid rgb(0, 0, 0);">bq.data.px_last(side='bid')</td>
        </tr>
        <tr>
            <td style="width: 12.5%; border: solid rgb(0, 0, 0);">Spread Type</td>
            <td style="width: 12.5%; border: solid rgb(0, 0, 0);">SPREAD_TYPE</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Z</td>
            <td style="width: 35%; border: solid rgb(0, 0, 0);">I, Z, ASW, OAS, BMK, G</td>
            <td style="width: 25%; border: solid rgb(0, 0, 0);">bq.data.spread(spread_type='z')</td>
        </tr>
        <tr>
            <td style="width: 12.5%; border: solid rgb(0, 0, 0);">Yield Type</td>
            <td style="width: 12.5%; border: solid rgb(0, 0, 0);">YIELD_TYPE</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">YTM</td>
            <td style="width: 35%; border: solid rgb(0, 0, 0);">YTM,YTW</td>
            <td style="width: 25%; border: solid rgb(0, 0, 0);">bq.data.yield_(yield_type='ytw')</td>        
        </tr>
    </tbody>
</table>



For more data items, you can run FLDS on the terminal to find available fields compatible in BQuant

---

<a id="bql-for-fi-querying-data" />


#### How to get Ask side Yield to Maturity (YTM) with BGN pricing source?

In [31]:
# Set the universe as an individual bond, identified by its Bloomberg ticker
universe = 'ZS7759443 Corp'

# Query BQL for the bond's short name
data_item = bq.data.yield_(yield_type='YTM',pricing_source='BGN',side='ASK')

# Build and execute the BQL query
request = bql.Request(universe, data_item)
response = bq.execute(request)

# Display the data as a Pandas DataFrame
data = response[0].df()
data

,DATE,"YIELD(side='ASK',yield_type='YTM',pricing_source='BGN')"
ID,,
ZS7759443 Corp,2024-09-12,2.918824


#### How to get Z Spread using BGN pricing source, but if BGN is not available, use BVAL pricing source as alternative.

In [32]:
ticker =  'ZS7759443 Corp'

bgn = bq.data.spread(spread_type='Z',pricing_source='BGN')
bval = bq.data.spread(spread_type='Z',pricing_source='BVAL')
data = bgn.avail(bval)

req = bql.Request(ticker,data)
res = bq.execute(req)
res[0].df()

,DATE,"AVAIL(SPREAD(spread_type='Z',pricing_source='BGN'),SPREAD(spread_type='Z',pricing_source='BVAL'))"
ID,,
ZS7759443 Corp,2024-09-12,68.013271


### Other bond data and parameters
<table style="width: 100%; border-collapse: collapse; border: solid rgb(0, 0, 0);">
    <tbody>
        <tr>
            <td style="width: 50%; border: 3px solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Name</span></strong></div>
            <td style="width: 50%; border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Mnenomic</span></strong></div> 
        </tr> 
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">CORP Ticker</td>
            <td style="width: 30%; border: solid rgb(0, 0, 0);">TICKER</td>
        </tr>  
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Country of Risk</td>
            <td style="width: 30%; border: solid rgb(0, 0, 0);">CNTRY_OF_RISK</td>
        </tr>   
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Currency</td>
            <td style="width: 30%; border: solid rgb(0, 0, 0);">CRNCY</td>
        </tr>   
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Coupon</td>
            <td style="width: 30%; border: solid rgb(0, 0, 0);">CPN</td>
        </tr>   
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Coupon Type</td>
            <td style="width: 30%; border: solid rgb(0, 0, 0);">CPN_TYP</td>
        </tr>  
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Collateral Type</td>
            <td style="width: 30%; border: solid rgb(0, 0, 0);">COLLAT_TYP</td>
        </tr>  
       <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Maturity Type</td>
            <td style="width: 30%; border: solid rgb(0, 0, 0);">MTY_TYP</td>
        </tr>  
       <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Payment Rank</td>
            <td style="width: 30%; border: solid rgb(0, 0, 0);">PAYMENT_RANK</td>
        </tr>  
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Maturity</td>
            <td style="width: 30%; border: solid rgb(0, 0, 0);">MATURITY</td>
        </tr> 
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Issue Date</td>
            <td style="width: 30%; border: solid rgb(0, 0, 0);">ISSUE_DT</td>
        </tr>  
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Issue Price</td>
            <td style="width: 30%; border: solid rgb(0, 0, 0);">ISSUE_PX</td>
        </tr>  
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Amount Issued</td>
            <td style="width: 30%; border: solid rgb(0, 0, 0);">AMT_ISSUED</td>
        </tr>   
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Amount Outstanding</td>
            <td style="width: 30%; border: solid rgb(0, 0, 0);">AMT_OUTSTANDING</td>
        </tr>   
    </tbody>
</table>

Ratings Data Fields

<table style="width: 100%; border-collapse: collapse; border: solid rgb(0, 0, 0);">
    <tbody>
        <tr>
            <td style="width: 15%; border: 3px solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Name</span></strong></div>
            <td style="width: 25%; border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Mnenomic</span></strong></div>
            <td style="width: 60%; border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Overrides/Parameters</span></strong></div>
        </tr>
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Credit Rating</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">RATING</td>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">Can also be used for issuers. RATING_SOURCE='SANDP'/'MOODY'/'FITCH'</td>
        </tr>
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Outlook Rating</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">CREDIT_RATING＿OUTLOOK</td>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">Can also be used for issuers. CREDIT_RATING_SOURCE='SANDP'/'MOODY'/'FITCH'</td>
        </tr>
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">S&amp;P Rating</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">RTG_SP</td>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">Specific dates can be referenced. DATES='2022-12-31'</td>
        </tr>
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">S&amp;P Rating Watch</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">RTG_SP_WATCH</td>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">Specific dates can be referenced. DATES='2022-12-31'</td>
        </tr>        
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">S&amp;P Long Term Rating</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">RTG_SP_LONG</td>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">Specific dates can be referenced. DATES='2022-12-31'</td>
        </tr>        
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">FITCH Rating</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">RTG_FITCH</td>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">Specific dates can be referenced. DATES='2022-12-31'</td>
        </tr>
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">FITCH Rating Watch</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">RTG_FITCH_WATCH</td>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">Specific dates can be referenced. DATES='2022-12-31'</td>
        </tr>
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">FITCH Long Term Rating</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">RTG_FITCH_LONG</td>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">Specific dates can be referenced. DATES='2022-12-31'</td>
        </tr>
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">MOODY Rating</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">RTG_MOODY</td>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">Specific dates can be referenced. DATES='2022-12-31'</td>
        </tr>
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">MOODY Rating Watch</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">RTG_MOODY_WATCH</td>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">Specific dates can be referenced. DATES='2022-12-31'</td>
        </tr>
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">MOODY Long Term Rating (Issue Level)</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">RTG_MOODY_LONG_ISSUE_LEVEL</td>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">Specific dates can be referenced. DATES='2022-12-31'</td>
        </tr>
        <tr>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">Bloomberg Composite</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">BB_COMPOSITE</td>
            <td style="width: 70%; border: solid rgb(0, 0, 0);">Blend of a security's Moody's, S&amp;P、Fitch and DBRS ratings. The rating agencies are evenly weighted when calculating the composite. It is calculated by taking the average of the existing ratings, rounded down to the lower rating in case the composite is between two ratings. <br>From this, the Bloomberg Composite Rating IG HY Indicator is derived. A composite will not be generated if the bond is rated by only one of the four rating agencies. <br>Specific dates can be referenced. DATES='2022-12-31'</td>
        </tr>
    </tbody>
</table>




---

<a id="bql-for-fi-querying-data" />


### Querying for multiple data items

To query for several data items at once, build your request with a dictionary of the data items. This example constructs a BQL query to return five separate data items about a single bond.


In [33]:
# Set the universe as an individual bond, identified by its FIGI
universe = 'BBG00NWNF0K4'

# Create a dictionary with multiple data items:
# - Each key is your custom name for the data field
# - Each value is a BQL data item
data_items = {
    'Yield' : bq.data.yield_(),
    'Issue Date' : bq.data.issue_date(),
    'Maturity' : bq.data.maturity(),
    'Issuer' : bq.data.company_corp_ticker(),
    'Payment Rank' : bq.data.payment_rank(),
    'Country of Risk' : bq.data.cntry_of_risk()
}

# Build the query and execute the request
request = bql.Request(universe, data_items)
response = bq.execute(request)

Use `pandas` to concatenate all the returned data items into a single dataframe:

In [34]:
data = pd.concat(
    [data_item.df()[data_item.name] for data_item in response], 
    axis=1
)
data

,Yield,Issue Date,Maturity,Issuer,Payment Rank,Country of Risk
ID,,,,,,
BBG00NWNF0K4,4.956015,2019-05-13,2046-02-01,ABIBB,Sr Unsecured,BE


In [35]:
# Calling out the response
response[0]

In [36]:
# Calling out the response for the data item into a data frame, output includes the underlying meta data 
response[0].df()

,DATE,Yield
ID,,
BBG00NWNF0K4,2024-09-12,4.956015


In [37]:
# Pulling out only the 'Price' value
response[0].df()['Yield']

ID
BBG00NWNF0K4    4.956015
Name: Yield, dtype: float64

In [38]:
# Concatenate the response for Name and Price into a dataframe using pandas
pd.concat([response[0].df()['Yield'],response[1].df()['Issue Date']],axis=1)

,Yield,Issue Date
ID,,
BBG00NWNF0K4,4.956015,2019-05-13


In [39]:
# Adding the other data items, however not the most efficient
pd.concat([response[0].df()['Yield'],response[1].df()['Issue Date'],response[2].df()['Maturity'],response[3].df()['Issuer'],response[4].df()['Payment Rank'],response[5].df()['Country of Risk']],axis=1)

,Yield,Issue Date,Maturity,Issuer,Payment Rank,Country of Risk
ID,,,,,,
BBG00NWNF0K4,4.956015,2019-05-13,2046-02-01,ABIBB,Sr Unsecured,BE


In [40]:
# Using list comprehension to concatenate into a dataframe
pd.concat([data_item.df()[data_item.name] for data_item in response],
          axis=1)

,Yield,Issue Date,Maturity,Issuer,Payment Rank,Country of Risk
ID,,,,,,
BBG00NWNF0K4,4.956015,2019-05-13,2046-02-01,ABIBB,Sr Unsecured,BE


---

<a id="bql-for-fi-querying-multiple" />


### Querying for multiple bonds

If you want to analyze more than one bond at a time, define a universe of bonds by providing BQL with a list of identifiers.

You can use any combination of Bloomberg tickers, CUSIPs, ISINs, FIGIs to identify the securities in your universe of interest.


In [41]:
# Set the universe as multiple bonds
universe = [
    'EJ729042 Corp',    # Bloomberg Ticker
    'US92556HAE71',     # ISIN
    '92556HAB3',        # CUSIP
    'BBG00G38H326',     # FIGI
]

# Define the target data
data_items = {
    'Bond Name' : bq.data.name(),
    'OAS' : bq.data.spread(spread_type='OAS'),
    'Credit Rating' : bq.data.bb_composite()
}

# Build and execute the request, and display the results
request = bql.Request(universe, data_items)
response = bq.execute(request)
data = pd.concat([field.df()[field.name] for field in response], axis=1)
data


,Bond Name,OAS,Credit Rating
ID,,,
EJ729042 Corp,PARA 4 ⅜ 03/15/43,360.056793,BB+
US92556HAE71,PARA 6 ⅜ 03/30/62,513.773360,BB
92556HAB3,PARA 4.95 01/15/31,291.429941,BB+
BBG00G38H326,PARA 6 ¼ 02/28/57,509.416000,BB


---

<a id="bql-for-fi-querying-dates" />


### Specifying dates

By default, BQL queries return the most up-to-date data available. You can get historical data by specifying either a relative date (e.g., `'-1W'`) or an absolute date (e.g., `'2022-12-07'`) with the `dates` parameter. This example returns price of a Vodafone Bond as of 7 December 2022.


<a></a>
<div style="background-color: #353535; white-space: nowrap; padding-top: 0.5rem;">
    <p style="margin-top: -10px; margin-bottom: 0px; margin-left: 5px; padding: 7px; overflow: hidden;">
        <span style="color: white; font-size: 1em; font-weight: bold;"><span style="color: rgb(0, 200, 82);"><strong>&#x24d8;</strong></span>&nbsp;&nbsp;Tip </span>
    </p>
</div>
<div style="background-color: #232323; white-space: nowrap; margin-bottom: 0.5rem;">
    <p style="margin-bottom: 0px; margin-left: 5px; padding: 7px; overflow: hidden;">To give an absolute date, use the format <code>YYYY-MM-DD</code> in a string.
    </p>
</div>

In [42]:
# Set the universe as an individual bond
universe = 'ZS7759443 Corp'

# Set the date
# The date format is YYYY-MM-DD
date = '2022-12-07'

# Query BQL the last price of a bond
price = bq.data.px_last(dates=date)

# Build and execute the request, and display the results
request = bql.Request(universe, price)
response = bq.execute(request)
data = response[0].df()
data

,DATE,CURRENCY,PX_LAST(dates=2022-12-07)
ID,,,
ZS7759443 Corp,2022-12-07,None,89.176


---

<a id="bql-for-fi-querying-time-series" />


### Retrieving time series data

Use [`func.range()`](https://help.bloomberg-quant.com/bql/functions/range) to query BQL for time series data over a range of dates. This example pulls the price of the bond (with [`data.px_last()`](https://help.bloomberg-quant.com/bql/data-items/px_last)) on each date from 1 December to 14 December 2023.

**Note:** This example uses [`dropna()`](https://help.bloomberg-quant.com/bql/functions/dropna) to filter out unavailable data points (e.g., days when the markets are closed).


In [43]:
# Set the universe as an individual bond
universe = 'ZS7759443 Corp'

# Set the date range
date_range = bq.func.range('2023-12-01','2023-12-14')

# Query BQL for the last price and drop rows that do not have a value
px_series = bq.data.px_last(dates=date_range).dropna()

# Build and execute the request, and display the results
request  = bql.Request(universe, px_series)
response = bq.execute(request)
data = response[0].df()
data.head()

,DATE,CURRENCY,"DROPNA(PX_LAST(dates=RANGE(2023-12-01,2023-12-14)))"
ID,,,
ZS7759443 Corp,2023-12-01,None,88.626
ZS7759443 Corp,2023-12-04,None,88.705
ZS7759443 Corp,2023-12-05,None,89.179
ZS7759443 Corp,2023-12-06,None,89.393
ZS7759443 Corp,2023-12-07,None,89.491


You can also specify date ranges using relative dates. This example pulls a bond's yield for the past seven days, and calculates the percentage change in its yield over that time.

**Note:** This example uses `fill='prev'` to replace any missing yield data with the last available value.


In [44]:
# Set the universe as an individual bond
universe = 'ZS7759443 Corp'

# Set the relative date 
last_week = bq.func.range('-1w','0d')

# Query BQL for yield percent change for the last week and fill in missing data 
yld_chg = bq.data.yield_(dates=last_week, fill='prev').pct_chg()

# Build and execute the request, and display the results
request = bql.Request(universe, {'Yield Change': yld_chg})
response = bq.execute(request)
data = response[0].df()
data

,DATE,Yield Change
ID,,
ZS7759443 Corp,2024-09-12,-2.826981


---

<a id="bql-for-fi-univ" />


## Building universes with `univ`

Rather than specifying lists of individual bonds, you can quickly build universes of relevant instruments with BQL's [`univ`](https://help.bloomberg-quant.com/content?id=XcGc6XrcLwzzUDzXyxwFcz) accessor property:

- Use [`univ.bondsuniv()`](https://help.bloomberg-quant.com/bql/universe_functions/bondsuniv) to pull all available bonds. You might want to call this method with `'active'` as a parameter to exclude all matured bonds.

- Use [`univ.bonds()`](https://help.bloomberg-quant.com/bql/universe_functions/bonds) to pull bonds for one or more specific issuers. You can specify issuers using equity, loan, corporate bond, or government bond IDs. The accessory property accepts a single ID or a list of IDs.

- Use [`univ.loans()`](https://help.bloomberg-quant.com/bql/universe_functions/loans) to pull loans.

- Use [`univ.debt()`](https://help.bloomberg-quant.com/bql/universe_functions/debt) to pull bonds, loans, US preferreds, and munis.


---

<a id="bql-for-fi-univ-from-list" />


### Building a universe from a list of bonds, loans or debt

The following example builds a universe of bonds/loans/debt instruments issued by Apple and Microsoft, then queries BQL for the OAS spread for each bond.


In [45]:
# Set the universe as bond instruments issued by a list of equities
universe = bq.univ.bonds(['AAPL US Equity', 'IBM US Equity'])

# Query BQL for the bond name, issuer, and OAS
data_items = {
    'Bond Name' : bq.data.name(),
    'Issuer' : bq.data.issuer(),
    'OAS' : bq.data.spread(spread_type='OAS')
}
    
# Build and execute the request, and display the results
request = bql.Request(universe, data_items)
response = bq.execute(request)
data = pd.concat([field.df()[field.name] for field in response], axis=1)
data

,Bond Name,Issuer,OAS
ID,,,
ZK536546 Corp,AAPL 4.85 05/10/53,APPLE INC,140.283362
JK138055 Corp,AAPL 4.65 02/23/46,APPLE INC,137.080347
BK930045 Corp,AAPL 2.55 08/20/60,APPLE INC,132.565718
JK138046 Corp,AAPL 3 ¼ 02/23/26,APPLE INC,19.450981
EK900686 Corp,AAPL 4 ⅜ 05/13/45,APPLE INC,137.751866
...,...,...,...
BU124785 Corp,IBM 1 ¼ 02/09/34,IBM CORP,90.622707
BU144753 Corp,IBM 2.2 02/09/27,IBM CORP,71.834833
JK286291 Corp,IBM 1 ¾ 03/07/28,IBM CORP,64.080788


In [46]:
# Set the universe as loans instruments issued by a list of equities
universe = bq.univ.loans(['AAPL US Equity', 'IBM US Equity'])

# Query BQL for the bond name, issuer, and OAS
data_items = {
    'Bond Name' : bq.data.name(),
    'Issuer' : bq.data.issuer(),
}
    
# Build and execute the request, and display the results
request = bql.Request(universe, data_items)
response = bq.execute(request)
data = pd.concat([field.df()[field.name] for field in response], axis=1)
data

,Bond Name,Issuer
ID,,
NULL_ID,None,None
BL304477 Corp,APTI TL 1L USD Corp,APPTIO INC
BL365829 Corp,IBM REV UNSEC USD Corp,IBM CORP
BL365834 Corp,IBM REV GUAR USD Corp,IBM CORP
BL422927 Corp,APTI TL 1L USD Corp,APPTIO INC
BL453987 Corp,APTI REV 1L USD Corp,None


In [47]:
# Set the universe as debt instruments issued by a list of equities
universe = bq.univ.debt(['AAPL US Equity', 'IBM US Equity'])

# Query BQL for the bond name, issuer, and OAS
data_items = {
    'Bond Name' : bq.data.name(),
    'Issuer' : bq.data.issuer(),
    'OAS' : bq.data.spread(spread_type='OAS')
}
    
# Build and execute the request, and display the results
request = bql.Request(universe, data_items)
response = bq.execute(request)
data = pd.concat([field.df()[field.name] for field in response], axis=1)
data

,Bond Name,Issuer,OAS
ID,,,
ZK536546 Corp,AAPL 4.85 05/10/53,APPLE INC,140.283362
JK138055 Corp,AAPL 4.65 02/23/46,APPLE INC,137.080347
BK930045 Corp,AAPL 2.55 08/20/60,APPLE INC,132.565718
JK138046 Corp,AAPL 3 ¼ 02/23/26,APPLE INC,19.450981
EK900686 Corp,AAPL 4 ⅜ 05/13/45,APPLE INC,137.751866
...,...,...,...
BL304477 Corp,APTI TL 1L USD Corp,APPTIO INC,NaN
BL365829 Corp,IBM REV UNSEC USD Corp,IBM CORP,NaN
BL365834 Corp,IBM REV GUAR USD Corp,IBM CORP,NaN


More parameters of bq.univ.bonds() / bq.univ.loans() / bq.univ.debt() will be covered in session 2 of FI series, example - issued by capital structure 

---

<a id="bql-for-fi-univ-from-basket" />


### Building a universe from an existing basket of securities

Use [`univ.members()`](https://help.bloomberg-quant.com/bql/universe_functions/members) to decompose an existing basket of securities (e.g., an index, portfolio, or region) into its constituent members.


In [48]:
# Set the universe as the members of the Bloomberg US Corporate Bond Index
universe = bq.univ.members('LUACTRUU Index')
data_item = bq.data.name()

# Build and execute the request, and display the results
request = bql.Request(universe, data_item)
response  = bq.execute(request)
data = response[0].df()
data.head()

,NAME()
ID,
EI198784 Corp,TRICN 5.85 04/15/40
QZ890029 Corp,CSX 3.8 11/01/46
QZ890464 Corp,ECL 2.7 11/01/26
EH277136 Corp,KR 6.9 04/15/38
EI464336 Corp,TWC 5 ⅞ 11/15/40


---

<a id="bql-for-fi-univ-cap-structure" />


### Determining index membership at a point in time

Since index membership changes over time, you can use the `dates` parameter with [`univ.members()`](https://help.bloomberg-quant.com/bql/universe_functions/members) to retrieve the members of an index on a certain date.

This example gets the members on an index as of 1 October 2023, then filters the universe by index members in the technology sector, and finally queries BQL for duration, convexity, and maturity data about the securities.


In [49]:
# Set the universe as members of an index as of a date
index = 'LUACTRUU Index'
as_of_date = '2023-10-01'
universe = bq.univ.members(index, dates=as_of_date)

# Filter by members in the refining sector
sector = bq.data.classification_name(classification_scheme='bclass', classification_level = '4')
bics3_filter = sector == 'Refining'
filtered_universe = bq.univ.filter(universe, bics3_filter)

# Query BQL for duration, convexity, maturity, and sector got a index member
duration = bq.data.duration()
convexity = bq.data.convexity(PRICING_SOURCE='BVAL')
maturity = bq.data.maturity().year()
data_items = {
  'Duration': duration,
  'Convexity': convexity,
  'Maturity': maturity,
  'Sector': sector
}

# Build and execute the request, and display the results 
request = bql.Request(filtered_universe, data_items)
response = bq.execute(request)    
data = pd.concat([x.df()[x.name] for x in response], axis=1)
data.head()

,Duration,Convexity,Maturity,Sector
ID,,,,
AR495387 Corp,3.061398,NaN,2028,Refining
AS711393 Corp,3.226779,NaN,2028,Refining
AX804131 Corp,3.929526,NaN,2029,Refining
BH634654 Corp,0.530336,NaN,2025,Refining
BJ097735 Corp,0.554357,NaN,2025,Refining


---

<a id="bql-for-fi-univ-import" />


### Importing private portfolios from the Bloomberg Terminal®

You can use BQL to import your Terminal based portfolios -- including any portfolios you created with the Portfolio Administration function ([PRTU&lt;GO&gt;](https://blinks.bloomberg.com/screens/prtu)) or uploaded via the Bloomberg Uploader ([BBU&lt;GO&gt;](https://blinks.bloomberg.com/screens/bbu)).

To import a private portfolio, you'll need the portfolio's ID, which you can find in the upper-right corner of the Terminal window when viewing the portfolio in [PRTU&lt;GO&gt;](https://blinks.bloomberg.com/screens/prtu).

In this example, the `type='PORT'` argument imports private portfolio. Replace `'your_portfolio_id'` with a string containing your portfolio's ID.

In [50]:
# Use the type='PORT' keyword parameter to import a private portfolio
# Replace your_portfolio_id with a string of your portfolio id
portfolio = bq.univ.members('U14000284-202', type='PORT')

# Query BQL for the yield and weights for members in a private portfolio
yld = bq.data.yield_()
weights = bq.data.id()['WEIGHTS']
data_items = {
    'Yield' : yld,
    'Weights' : weights    
}

## Uncomment and run the following lines only after replacing 'your_portfolio_id' above
# request = bql.Request(portfolio, data_items)
# response = bq.execute(request)    
# data = pd.concat([x.df()[x.name] for x in response], axis=1)
# data.head()

In [51]:
request = bql.Request(portfolio, data_items)
response = bq.execute(request)

data = pd.concat(
     [field.df()[field.name] for field in response],
     axis=1
 )
data.head()

,Yield,Weights
ID,,
EK144038 Corp,5.956424,0.198059
AP657413 Corp,4.284235,0.377570
AP657424 Corp,5.217832,0.959326
AO080878 Corp,5.897400,0.219596
UV340967 Corp,NaN,0.000000


---

<a id="bql-for-fi-filtering" />


## Aggregating, ranking, and sorting data


---

<a id="bql-for-fi-aggregating-group" />


### Performing aggregate analyses with `group()`

The [`func.group()`](https://help.bloomberg-quant.com/bql/functions/group) method lets you use BQL to partition your universe into subgroups, then perform aggregate operations over each subgroup.

Main functions used for aggregation

<table style="width: 100%; border-collapse: collapse; border: solid rgb(0, 0, 0);">
    <tbody>
        <tr>
            <td style="width: 50%; border: 3px solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Name</span></strong></div>
            <td style="width: 50%; border: solid rgb(0, 0, 0);">
                <div style="text-align: center;"><strong><span style="font-size: 16px;">Mnenomic</span></strong></div>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Count</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">COUNT()</td>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Average</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">AVG()</td>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Weighted Average</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">WAVG()</td>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Median</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">MEDIAN()</td>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Standard Deviation</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">STD()</td>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Max</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">MAX()</td>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Min</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">MIN()</td>
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Sum</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">SUM()</td> 
        </tr>
        <tr>
            <td style="width: 20%; border: solid rgb(0, 0, 0);">Bins</td>
            <td style="width: 15%; border: solid rgb(0, 0, 0);">BINS()</td> 
        </tr>
    </tbody>
</table>





#### This example calculates the median OAS per maturity year for a universe of bonds that were issued by the pharmaceutical companies Novartis, Roche, and GSK.


In [52]:
# Set the universe as bond chains for a list of issuers
universe = bq.univ.bonds(['NOVN SW Equity','ROG SW Equity','GSK LN Equity'])

# Query BQL for OAS and maturity
oas_spread = bq.data.spread(spread_type='OAS')
maturity = bq.data.maturity().year()

# Get the OAS for every bond in the universe, group the bonds by maturity year, 
# and calculate the median OAS for each year
oas_spread_by_year = oas_spread.group(maturity).median()

data_item = {
    'Median OAS by Year (bp)' : oas_spread_by_year
            }             
             
# Build and execute the request, and display the results
request = bql.Request(universe, data_item)
response = bq.execute(request)
data = response[0].df()
data.head()

,DATE,ORIG_IDS,YEAR(MATURITY()),Median OAS by Year (bp)
ID,,,,
2024.0,2024-09-12,AM786418 Corp,2024,53.979817
2025.0,2024-09-12,None,2025,25.444598
2026.0,2024-09-12,None,2026,23.747846
2027.0,2024-09-12,None,2027,36.968953
2028.0,2024-09-12,None,2028,53.281071


#### How to find the count of bonds by BCLASS classification of I00163JP Index (Asia Pacific Aggregate Index)

In [53]:
# Set the universe for the index members of Asia-Pac Aggregate (over 2000 members)
universe = bq.univ.members('I00163JP index')

# Query BQL for sector and id
sector = bq.data.classification_name(classification_scheme='BCLASS',classification_level='2')
num_id = bq.data.id().group(sector).count()

# Get the bond id in the universe, group the bonds by maturity year, 
# and calculate the count of bond per sector
data_item = {
    'Count' : num_id
            }    

# Build and execute the request, and display the results
request = bql.Request(universe, data_item,with_params=dict(mode='cached'))
response = bq.execute(request)
data = response[0].df()
data.head()

,Weights,Positions,ORIG_IDS:0,ORIG_IDS:1,"CLASSIFICATION_NAME(CLASSIFICATION_SCHEME='BCLASS',CLASSIFICATION_LEVEL='2')",Count
ID,,,,,,
Agency,NaN,NaN,None,I00163JP Index,Agency,630
Covered,NaN,NaN,None,I00163JP Index,Covered,7
Financial Institutions,NaN,NaN,None,I00163JP Index,Financial Institutions,226
Industrial,NaN,NaN,None,I00163JP Index,Industrial,164
Local Authority,NaN,NaN,None,I00163JP Index,Local Authority,352


More examples of functions can be found on BQLX

---

<a id="bql-for-fi-aggregating-rating" />


## Conclusion


Now that you've seen a few of the ways to use PyBQL in BQuant to explore and analyze Bloomberg's fixed-income datasets, we recommend going through the [BQL Basics Tutorial](https://help.bloomberg-quant.com/content?id=QsPuFyBLPMmiH3fwgkd4EP) for a more thorough introduction to the design and function of PyBQL.
